In [1]:
import scipy.io as sc
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch, butter, lfilter
from scipy.signal.windows import hamming
import nibabel as nib
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import minmax_scale

In [9]:
# Import all custom functions
from custom_functions import (
    merge_rois,
    implant2roi,
    get_norm_psd,
    get_norm_entropy,
    plot_ieeg_atlas,
    compare_ieeg_atlas,
    make_seizure_free,
    make_seizure_free_abr,
    univariate_abr,
    node_abr_edge,
    edgeslist_to_abr_conn,
    main_univar
)

ImportError: cannot import name 'merge_rois' from 'custom_functions' (/Users/tereza/nishant/atlas/atlas_work_terez/atlas_harmonization/code/custom_functions.py)

In [2]:
base_path = '/Users/tereza/nishant/atlas/epi_iEEG_atlas'

In [ ]:
def main():
    # Parameters
    EngelSFThres = 1.1
    spikeThresh = 24

    # Load and prepare atlas data
    print("Loading atlas data...")
    atlas = {}
    atlas_path = os.path.join(base_path, 'Data', 'AAL.nii.gz')
    atlas['data'] = nib.load(atlas_path).get_fdata()
    atlas['data'][atlas['data'] > 9000] = 0
    atlas['hdr'] = nib.load(atlas_path).header
    
    # Load other required data
    print("Loading other data files...")
    MNI_atlas = sc.loadmat(os.path.join(base_path, 'Data', 'MNI_atlas.mat'))
    HUP_atlas = sc.loadmat(os.path.join(base_path, 'Data', 'HUP_atlas.mat'))
    metaData = sc.loadmat(os.path.join(base_path, 'Data', 'metaData.mat'))
    customAAL = pd.read_excel(os.path.join(base_path, 'Data', 'custom_atlas.xlsx'))
    roiAAL = sc.loadmat(os.path.join(base_path, 'Data', 'roiAAL.mat'))

    # Create custom atlas
    print("Creating custom atlas...")
    atlasCustom, roiAALcustom = merge_rois(customAAL, roiAAL, atlas)
    atlas['tbl'] = roiAAL
    atlasCustom['tbl'] = roiAALcustom
    atlas = atlasCustom

    # Run univariate analysis
    print("Running univariate analysis...")
    norm_MNI_HUP_Atlas, iEEG_hup_all, abrnorm_hup_atlas = main_univar(
        metaData, atlas, MNI_atlas, HUP_atlas, EngelSFThres, spikeThresh
    )

    # Generate univariate z-scores
    print("Computing univariate z-scores...")
    iEEG_hup_all_z = univariate_abr(norm_MNI_HUP_Atlas, iEEG_hup_all)

    # Create output directory if it doesn't exist
    os.makedirs(os.path.join(base_path, 'Figures'), exist_ok=True)

    # Generate and save comparison plots
    print("Generating comparison plots...")
    plot_comparisons(norm_MNI_HUP_Atlas, iEEG_hup_all_z, base_path)

    return norm_MNI_HUP_Atlas, iEEG_hup_all_z, abrnorm_hup_atlas

def plot_comparisons(norm_MNI_HUP_Atlas, iEEG_hup_all_z, base_path):
    """Generate comprehensive comparison plots"""
    
    # Plot 1: Feature distributions across cohorts
    plt.figure(figsize=(12, 6))
    bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    plt.boxplot([iEEG_hup_all_z[f'{band}_z'] for band in bands])
    plt.xticks(range(1, len(bands) + 1), bands)
    plt.ylabel('Z-scored Power')
    plt.title('Distribution of Frequency Band Powers')
    plt.savefig(os.path.join(base_path, 'Figures', 'band_powers.pdf'))
    plt.close()

    # Plot 2: ROI comparison
    plt.figure(figsize=(15, 8))
    roi_means = norm_MNI_HUP_Atlas.groupby('roi')[bands].mean()
    roi_means.plot(kind='bar')
    plt.title('Mean Band Power by ROI')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(base_path, 'Figures', 'roi_comparison.pdf'))
    plt.close()

# If running in a notebook, you can run this directly:
try:
    norm_MNI_HUP_Atlas, iEEG_hup_all_z, abrnorm_hup_atlas = main()
    print("Analysis completed successfully.")
    print(f"Results saved in {os.path.join(base_path, 'Figures')}")
except Exception as e:
    print(f"Error during analysis: {str(e)}")